In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None
%config SqlMagic.named_parameters="enabled"

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
from typing import Generator
import requests
import bs4
import datetime

def get_vacancy(term: str) -> Generator[dict, None, None]:
    next_page = True
    url = "https://www.jobs.ch"
    uri = "/en/vacancies/"
    headers = {"User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36"}
    while next_page is not None:
        response = requests.get(url + uri, headers=headers) if (type(next_page) != bool) else requests.get(url + uri, params={"term": term}, headers=headers)
        soup = bs4.BeautifulSoup(response.text, "html.parser")
        tag = soup.find(attrs={"data-cy": "vacancy-serp-item"})
        while tag is not None:
            items = [item.string for item in tag.select("p,span") if item.string is not None and item.string not in ["Recommended", "Quick Apply"]]
            if len(items) == 7:
                yield dict(
                    id=tag.parent.get("href").split("/")[-2],
                    publish_date=datetime.datetime.strptime(items[0].replace("Published: ", ""), "%d %B %Y").date().strftime("%Y-%m-%d"),
                    title=items[2],
                    location=items[3],
                    quota=items[4],
                    employment_type=items[5],
                    company=items[6],
                    link=f'{url}{tag.parent.get("href").split("?")[0]}')
            tag = tag.find_next(attrs={"data-cy": "vacancy-serp-item"})
        if (next_page := soup.find("a", attrs={"data-cy": "paginator-next"})) is not None:
            uri = next_page.get("href")

In [8]:
%%sql sqlite:///jobs.db
DROP TABLE IF EXISTS jobs;
CREATE TABLE jobs (id text primary key, publish_date date, title text, company text, location text, employment_type text, quota text, link text);

++
||
++
++

In [9]:
%%capture
for row in get_vacancy("Data Engineer"):
    id, publish_date, title, location, quota, employment_type, company, link = tuple(row.values())
    try:
        %sql INSERT INTO jobs (id, publish_date, title, company, location, employment_type, quota, link) VALUES (:id, :publish_date, :title, :company, :location, :employment_type, :quota, :link);
    except Exception as e:
        print(f"Error: {e} - Skipping row with id {id}")

In [10]:
%sql SELECT count(*) FROM jobs;

Running query in 'sqlite:///jobs.db'

count(*)
1543


In [12]:
%%sql sqlite:///jobs.db
SELECT * FROM jobs LIMIT 5;

id,publish_date,title,company,location,employment_type,quota,link
fb32064a-1974-400a-908a-0ce286e0bcfc,2024-10-21,SHE Ingenieur (m/w/d),DSM-Firmenich,Lalden,Unlimited employment,100%,https://www.jobs.ch/en/vacancies/detail/fb32064a-1974-400a-908a-0ce286e0bcfc/
98a8c832-5083-47c8-9e0d-1e5299e1c159,2024-10-01,Ingenieur/in für weltweite Inbetriebnahmen,Maerz Ofenbau AG,Zürich,Unlimited employment,100%,https://www.jobs.ch/en/vacancies/detail/98a8c832-5083-47c8-9e0d-1e5299e1c159/
a91ceb8f-6ba3-4bde-8dd7-ca096156037f,2024-10-17,Senior FPGA Engineer (f/m/d),Leica Geosystems AG,Heerbrugg,Unlimited employment,100%,https://www.jobs.ch/en/vacancies/detail/a91ceb8f-6ba3-4bde-8dd7-ca096156037f/
906c8e08-ea42-4074-93eb-d44af37e9855,2024-10-15,DevOps Engineer C2I,RUAG MRO Holding AG,Thun,Unlimited employment,80 – 100%,https://www.jobs.ch/en/vacancies/detail/906c8e08-ea42-4074-93eb-d44af37e9855/
90b4e99e-2441-4bf0-a3d5-ef2d0cbff59a,2024-10-08,Junior Technical Engineer 80 - 100% (m/w),BIBUS AG,Fehraltorf,Unlimited employment,80 – 100%,https://www.jobs.ch/en/vacancies/detail/90b4e99e-2441-4bf0-a3d5-ef2d0cbff59a/
